# Training Policy

##### This module deals with training any policy and serialize and store the model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import datetime
import os
import tensorflow as tf
from utils import train_model

In [3]:
from dotenv import dotenv_values
env_vars = dotenv_values('.env')

In [4]:
start = datetime.datetime(2025, 5, 20, 9, 30, 0)
end = datetime.datetime(2025, 5, 20, 14, 30, 0)
headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": env_vars["ALPACA_API_KEY"],
    "APCA-API-SECRET-KEY": env_vars["ALPACA_API_SECRET"],
}
company = "TSLA"
model_name = "tardoBot_v2.0"
model_save_path = "../models/tradoBot_v2.0"
log_file_path = "../model_logs/tradoBot_v2.0_logs.txt"
img_folder = "../model_logs/training_graphs"

In [5]:
# data_features = number of input features after data processsing = len(policy.data_features)
data_features = 6
policy = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(data_features,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])
policy.data_features = (["c-o/o", "h-l/o", "h-o/o", "o-l/o", "c", "vol"])  # feature names

In [6]:
print("Weights shape of each layer:")
for layer in policy.layers:
    weights = layer.get_weights()
    print(f"{layer.name}: {[w.shape for w in weights]}")

Weights shape of each layer:
dense: [(6, 64), (64,)]
dense_1: [(64, 64), (64,)]
dense_2: [(64, 3), (3,)]


6. Train Model

In [7]:
train_model(
    policy=policy,
    start_date=start,
    end_date=end,
    headers=headers,
    company=company,
    log_file=log_file_path,
    img_folder=img_folder,
    get_log=True,
    sleep_time=3,)

Log file ../model_logs/tradoBot_v2.0_logs.txt does not exist, creating a new one.
from 2025-05-20 09:30:00 for 90mins of TSLA => reward : max : 11.550,avg : 1.644, final : 8.991
from 2025-05-20 11:00:00 for 79mins of TSLA => reward : max : 3.933,avg : 0.380, final : 0.000
from 2025-05-20 12:19:00 for 77mins of TSLA => reward : max : 34.935,avg : 3.412, final : 34.399
from 2025-05-20 13:36:00 for 82mins of TSLA => reward : max : 11.128,avg : -1.615, final : 0.000


In [8]:
file = open(model_save_path,'wb')
pickle.dump(policy,file)
file.close()